# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [35]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize


In [36]:
# Paste the url you obtained for your data
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale'
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}


#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions). Hint: use the headers parameter to get the data!

In [37]:
# headers definition
response = requests.get(url,headers=headers)
response

<Response [200]>

In [38]:
# Your code
result=response.json()
result
data=pd.DataFrame(json_normalize(result))
data.head()



,total_count,sort,articles,query_path,previous_page_path,next_page_path,page_gender,premium,filters,total_article_count,...,iconPaths.filters.standard_delivery_filter,iconPaths.filters.fast_delivery_filter,iconPaths.filters.zalando_plus,iconPaths.mobileFilters.standard_delivery_filter,iconPaths.mobileFilters.fast_delivery_filter,iconPaths.mobileFilters.zalando_plus,iconPaths.flags.slow_delivery_flag,iconPaths.flags.fast_delivery_flag,iconPaths.flags.plus_delivery_flag,iconPaths.flags.zalando_plus
0,22059,sale,"[{'sku': 'NI114D0AV-A11', 'name': 'AIR MAX 200...",/promo-enfant/?p=2&order=sale,/promo-enfant/?order=sale,/promo-enfant/?p=3&order=sale,kids,False,"[{'key': 'sizes', 'label': 'Taille', 'url_key'...",22066,...,icons/truck.svg,icons/truck-fast.svg,icons/plus-short-1.svg,icons/truck.svg,icons/truck-fast.svg,icons/plus-short-1.svg,icons/clock.svg,icons/truck-fast-orange-3.svg,icons/plus-short-1.svg,icons/zalando-plus.svg


In [39]:
data2=json_normalize(data['articles'][0])
data2.head(2)

,sku,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,amount
0,NI114D0AV-A11,AIR MAX 200 - Baskets basses - white/black/bri...,"[17, 18.5, 19.5, 21, 22, 23.5, 25]",nike-sportswear-air-max-200-baskets-basses-ni1...,[{'path': 'NI/11/4D/0A/VA/11/NI114D0AV-A11@11....,Nike Sportswear,False,"[{'sku': 'NI114D0AV-A11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': 'Jusqu’à -15...",shoe,[],"59,95 €","50,95 €",True,False,True,False,NaN
1,NA824O01P-K11,NKMSNOW03 JACKET BLOCK - Veste d'hiver - dark...,"[8a, 9a, 10a, 11a, 12a, 13a, 14a]",name-it-nkmsnow03-jacket-block-veste-dhiver-da...,[{'path': 'NA/82/4O/01/PK/11/NA824O01P-K11@4.j...,Name it,False,"[{'sku': 'NA824O01P-K11', 'url_key': 'name-it-...","[{'key': 'discountRate', 'value': 'Jusqu’à -60...",clothing,[],"49,99 €","19,99 €",True,False,True,False,NaN


#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

In [40]:
total_articles=result['total_count']
total_articles

22059

In [67]:
# Get the total number of pages
total_articles=result['total_count']
articles_page=result['pagination']['per_page']
calls_api=[i for i in range(84,total_articles+1,articles_page)]
calls_api.append(total_articles-calls_api[-1]+calls_api[-1])
final_result=pd.DataFrame()
for i in calls_api:
    if i%84==0:
        url=f'https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset={i}&sort=sale'
    else:
        x=calls_api[-1]-calls_api[-2]
        url=f'https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit={x}&offset={i}&sort=sale'        
    
    response = requests.get(url,headers=headers)
    result=response.json()
    data=pd.DataFrame(json_normalize(result))
    data2=json_normalize(data['articles'][0])
    final_result=final_result.append(data2)
    


In [68]:
final_result.shape

(21964, 19)

In [69]:
final_result.columns

Index(['amount', 'brand_name', 'delivery_promises', 'family_articles', 'flags',
       'is_premium', 'media', 'name', 'price.base_price',
       'price.has_different_original_prices', 'price.has_different_prices',
       'price.has_different_promotional_prices',
       'price.has_discount_on_selected_sizes_only', 'price.original',
       'price.promotional', 'product_group', 'sizes', 'sku', 'url_key'],
      dtype='object')

#### Display the trending brand in DataFrame

In [70]:
Brand = final_result.groupby('brand_name', as_index=False)['sku'].count()
Brand=Brand.rename(columns={"sku": "Num_Items"})
Brand.sort_values(by=['Num_Items'], ascending=False).head(10)


,brand_name,Num_Items
196,Name it,1021
96,GAP,717
83,Esprit,674
25,Boboli,669
15,Benetton,532
91,Friboo,523
213,OVS,512
206,Nike Sportswear,419
333,adidas Performance,417
204,Nike Performance,398


#### Display the brand with maximal total discount (sum of discounts on all goods)

In [71]:
final_result.head(2)

,amount,brand_name,delivery_promises,family_articles,flags,is_premium,media,name,price.base_price,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,sku,url_key
0,NaN,Name it,[],"[{'sku': 'NA824O01P-K11', 'url_key': 'name-it-...","[{'key': 'discountRate', 'value': 'Jusqu’à -60...",False,[{'path': 'NA/82/4O/01/PK/11/NA824O01P-K11@4.j...,NKMSNOW03 JACKET BLOCK - Veste d'hiver - dark...,NaN,False,True,True,False,"49,99 €","19,99 €",clothing,"[8a, 9a, 10a, 11a, 12a, 13a, 14a]",NA824O01P-K11,name-it-nkmsnow03-jacket-block-veste-dhiver-da...
1,NaN,Levi's®,[],"[{'sku': 'LE223A054-Q12', 'url_key': 'levisr-7...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'LE/22/3A/05/4Q/12/LE223A054-Q12@4.j...,710 SUPER SKINNY - Jeans Skinny - meteorite,NaN,False,True,True,False,"44,95 €","35,95 €",clothing,"[3a, 4a, 6a, 8a, 10a, 12a, 14a, 16a]",LE223A054-Q12,levisr-710-super-skinny-jeans-skinny-le223a054...


In [72]:
#Our data is still text. Convert prices into numbers.
# your code
final_result['price.original']=final_result['price.original'].replace('[\€,]', '', regex=True).astype(float)/100
final_result['price.promotional']=final_result['price.promotional'].replace('[\€,]', '', regex=True).astype(float)/100

final_result['discount']=final_result['price.original']-final_result['price.promotional']

final_result.head(2)



,amount,brand_name,delivery_promises,family_articles,flags,is_premium,media,name,price.base_price,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,sku,url_key,discount
0,NaN,Name it,[],"[{'sku': 'NA824O01P-K11', 'url_key': 'name-it-...","[{'key': 'discountRate', 'value': 'Jusqu’à -60...",False,[{'path': 'NA/82/4O/01/PK/11/NA824O01P-K11@4.j...,NKMSNOW03 JACKET BLOCK - Veste d'hiver - dark...,NaN,False,True,True,False,49.99,19.99,clothing,"[8a, 9a, 10a, 11a, 12a, 13a, 14a]",NA824O01P-K11,name-it-nkmsnow03-jacket-block-veste-dhiver-da...,30.0
1,NaN,Levi's®,[],"[{'sku': 'LE223A054-Q12', 'url_key': 'levisr-7...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'LE/22/3A/05/4Q/12/LE223A054-Q12@4.j...,710 SUPER SKINNY - Jeans Skinny - meteorite,NaN,False,True,True,False,44.95,35.95,clothing,"[3a, 4a, 6a, 8a, 10a, 12a, 14a, 16a]",LE223A054-Q12,levisr-710-super-skinny-jeans-skinny-le223a054...,9.0


In [73]:
Discounts = final_result.groupby('brand_name', as_index=False)['discount'].sum()
Products =final_result.groupby('brand_name', as_index=False)['sku'].count()
Agg_disc=Discounts.merge(Products,left_on='brand_name',right_on='brand_name')
Agg_disc=Agg_disc.rename(columns={"sku": "Num_Items"})
Agg_disc['avg_discount']=Agg_disc['discount']/Agg_disc['Num_Items']

Agg_disc.sort_values(by=['avg_discount'], ascending=False).head(10)

,brand_name,discount,Num_Items,avg_discount
186,Missoni Kids,450.00,4,112.500000
263,Schmuddelwedda,189.43,2,94.715000
149,Kjus,481.00,6,80.166667
281,Spyder,130.00,2,65.000000
318,Voile Blanche,61.50,1,61.500000
319,Volcom,59.00,1,59.000000
170,Lowa,767.00,14,54.785714
343,myMo,3054.55,56,54.545536
51,Colmar Originals,814.00,15,54.266667
27,Bomboogie,377.00,7,53.857143


#### Display the brands without discount at all

In [74]:
# your code
Agg_disc.sort_values(by=['avg_discount'], ascending=True).head(10)

,brand_name,discount,Num_Items,avg_discount
109,Havaianas,10.60,4,2.650000
335,camano,63.15,21,3.007143
62,DIM,75.20,24,3.133333
202,New Look 915 Generation,187.16,59,3.172203
84,Ewers,142.15,42,3.384524
273,Skip Hop,49.10,13,3.776923
68,Deuter,3.80,1,3.800000
28,Bonds,243.65,63,3.867460
331,Zip and Zoe,11.80,3,3.933333
201,New Era,189.10,48,3.939583
